<a href="https://colab.research.google.com/github/luiz826/Sistema_de_recomendacao/blob/main/Recomendation_Sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Recomendação para filmes da netflix

<img src="https://i2.wp.com/tvrev.com/wp-content/uploads/2020/02/thibault-penin-AWOl7qqsffM-unsplash-scaled.jpg?fit=2560%2C1709&ssl=1"> 

### Importação das bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/luiz826/Site_Sistema_de_recomendacao/main/netflix_titles.csv")

In [3]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


### Pré-processamento dos dados

##### Selecionando apenas os filmes e as colunas mais importantes

In [4]:
mov = df.query("type == 'Movie'")

In [5]:
movies = mov[["title", "director", "cast", "country", "rating", "listed_in"]].reset_index()
movies["title"] = movies["title"].apply(lambda x: x.lower())

##### Essa função serve para binarizar as palavras 

In [6]:
def binariza(col, classe):
    classe[col] = classe[col].fillna("Not Informed")
    
    lista = []
    for i in classe[col]:
        lista_aux = i.split(",")
        for j in lista_aux:
            lista.append(j)

    lista_sort = sorted(set(lista))

    binary_lista = [[0] * 0 for i in range(len(set(lista)))]

    for i in classe[col]:
        c = 0
        for j in lista_sort:
            if j in i:
                binary_lista[c].append(1.0)
            else:
                binary_lista[c].append(0.0)
            c+=1

    return pd.DataFrame(binary_lista).T

In [7]:
bin_actors = binariza("cast", movies)

In [8]:
bin_director = binariza("director", movies)

In [9]:
bin_country = binariza("country", movies)

In [10]:
bin_rating = binariza("rating", movies)

In [11]:
bin_listed_in = binariza("listed_in", movies)

##### Neste momento eu faço a concatenação das colunas e utilizo a similaridade de cossenos para encontrar a similaridade entre os binários

In [12]:
bin_df = pd.concat([bin_actors, bin_director, bin_country, bin_rating, bin_listed_in], axis=1, ignore_index=True)

In [13]:
cs_df = pd.DataFrame(cosine_similarity(bin_df))

### Definindo a função para gerar recomendações

In [26]:
def recomendation(movie, n=5):
    movie = movie.lower()

    if movie in movies["title"].values:
        idm = movies.loc[movies["title"] == movie].index.item()
        rm = cs_df[idm].sort_values(ascending=False).head(n+1)[1:]
        movies_c = movies.copy()
        movies_c["CS"] = cs_df[idm].sort_values(ascending=False)
        return movies_c.iloc[list(rm.index)]
    
    print("Esse filme não está na base de dados...\n")
    print("Ou você quis dizer: ", end="")
    for i in movies["title"].values:
        if movie in i:
            print(i)
    

##### Alguns exemplos

In [20]:
recomendation("Tom and Jerry: The Magic Ring", 7)

,index,title,director,cast,country,rating,listed_in,CS
2573,3672,little singham: kaal ki tabaahi,Prakash Satam,Not Informed,Not Informed,TV-Y7,"Children & Family Movies, Comedies",0.632456
2571,3670,little singham in london,Prakash Satam,Not Informed,Not Informed,TV-Y7,"Children & Family Movies, Comedies",0.632456
2574,3673,little singham: legend of dugabakka,Not Informed,Not Informed,Not Informed,TV-Y7,"Children & Family Movies, Comedies",0.632456
2569,3668,little singham aur kaal ka mahajaal,Prakash Satam,Not Informed,Not Informed,TV-Y7-FV,"Children & Family Movies, Comedies",0.603023
1508,2157,figaro pho,Luke Jurevicius,Luke Jurevicius,United Kingdom,TV-Y7,"Children & Family Movies, Comedies",0.553399
2575,3674,little singham: mahabali,Prakash Satam,"Saumya Daan, Sonal Kaushal, Anamaya Verma, Gan...",Not Informed,TV-Y7,"Children & Family Movies, Comedies",0.500000
2949,4241,motu patlu: king of kings,Suhas Kadav,"Saurav Chakrabarty, Vinay Pathak",India,TV-Y7-FV,"Children & Family Movies, Comedies",0.485363


In [21]:
recomendation("Pulp Fiction")

,index,title,director,cast,country,rating,listed_in,CS
3326,4776,paris is burning,Jennie Livingston,Not Informed,United States,R,"Classic Movies, Cult Movies, Documentaries",0.422116
131,165,a clockwork orange,Stanley Kubrick,"Malcolm McDowell, Patrick Magee, Michael Bates...","United Kingdom, United States",R,"Classic Movies, Cult Movies, Dramas",0.402492
940,1288,chasing amy,Kevin Smith,"Ben Affleck, Joey Lauren Adams, Jason Lee, Dwi...",United States,R,"Comedies, Cult Movies, Dramas",0.377124
2795,4008,mean streets,Martin Scorsese,"Robert De Niro, Harvey Keitel, David Proval, A...",United States,R,"Classic Movies, Dramas, Independent Movies",0.367065
5258,7609,who's that knocking at my door?,Martin Scorsese,"Zina Bethune, Harvey Keitel, Anne Collette, Le...",United States,R,"Classic Movies, Dramas, Independent Movies",0.367065


In [22]:
recomendation("The Two Popes", 2)

,index,title,director,cast,country,rating,listed_in,CS
1069,1501,cops and robbers,"Arnon Manor, Timothy Ware-Hill",Timothy Ware-Hill,United States,PG-13,Dramas,0.481125
1937,2794,honey: rise up and dance,Bille Woodruff,"Teyana Taylor, Sierra Aylina McClain, Bryshere...",United States,PG-13,Dramas,0.456435


# APP

In [33]:
#@title Aplicação
#@markdown Digite o nome do filme abaixo (em inglês)

Filme = 'pulp fiction' #@param {type:"string"}
Numero_de_Recomendacoes = 10 #@param {type:"number"}

In [34]:
recomendation(Filme, Numero_de_Recomendacoes)

,index,title,director,cast,country,rating,listed_in,CS
3326,4776,paris is burning,Jennie Livingston,Not Informed,United States,R,"Classic Movies, Cult Movies, Documentaries",0.422116
131,165,a clockwork orange,Stanley Kubrick,"Malcolm McDowell, Patrick Magee, Michael Bates...","United Kingdom, United States",R,"Classic Movies, Cult Movies, Dramas",0.402492
940,1288,chasing amy,Kevin Smith,"Ben Affleck, Joey Lauren Adams, Jason Lee, Dwi...",United States,R,"Comedies, Cult Movies, Dramas",0.377124
2795,4008,mean streets,Martin Scorsese,"Robert De Niro, Harvey Keitel, David Proval, A...",United States,R,"Classic Movies, Dramas, Independent Movies",0.367065
5258,7609,who's that knocking at my door?,Martin Scorsese,"Zina Bethune, Harvey Keitel, Anne Collette, Le...",United States,R,"Classic Movies, Dramas, Independent Movies",0.367065
4166,5994,taxi driver,Martin Scorsese,"Robert De Niro, Jodie Foster, Cybill Shepherd,...",United States,R,"Classic Movies, Dramas, Thrillers",0.341121
3521,5070,rain man,Barry Levinson,"Dustin Hoffman, Tom Cruise, Valeria Golino, Ge...",United States,R,"Classic Movies, Dramas",0.339550
4018,5770,spring breakers,Harmony Korine,"James Franco, Vanessa Hudgens, Selena Gomez, A...","United States, France",R,"Comedies, Cult Movies, Dramas",0.333623
878,1203,canvas,Frank E. Abney III,Not Informed,United States,G,"Children & Family Movies, Dramas",0.333333
4654,6722,the one i love,Charlie McDowell,"Mark Duplass, Elisabeth Moss, Ted Danson",United States,R,"Comedies, Dramas, Independent Movies",0.332820
